# UND ATSC Tools of the Trade

This code demonstrates how how read in netCDF data and do some basic data manipulation and plotting. 

Written: Aaron Scott, UND Fall 2022


In [ ]:
import pandas as pd
import numpy as np
import netCDF4 as nc
import matplotlib.pyplot as plt
import xarray as xr
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import warnings
warnings.filterwarnings('ignore')

In [ ]:
#Thanks to Brian Horan for this nifty function 
def set_background(ax):
    #ax.set_extent([255, 280, 40, 24])
    ax.coastlines('10m', edgecolor = 'black')
    ax.add_feature(cfeature.LAND.with_scale('10m'), facecolor = 'white')
    ax.add_feature(cfeature.BORDERS.with_scale('10m'))
    ax.add_feature(cfeature.LAKES.with_scale('10m'), edgecolor = 'black')
    ax.add_feature(cfeature.STATES.with_scale('10m'), edgecolor = 'black')
    ax.add_feature(cfeature.OCEAN.with_scale('10m'), edgecolor = 'black')
    #ax.add_feature(mpp.USCOUNTIES.with_scale('5m'))
    return ax

# Useful Links:
 - [Xarray Documentation](https://docs.xarray.dev/en/stable/index.html)
 - [Xarray Terminology](https://docs.xarray.dev/en/stable/user-guide/terminology.html)
 - [isel](https://docs.xarray.dev/en/stable/generated/xarray.Dataset.isel.html)
 - [Indexing and Selecting Data](https://docs.xarray.dev/en/stable/user-guide/indexing.html)
 - [FacetGrid](https://docs.xarray.dev/en/stable/generated/xarray.plot.FacetGrid.html)


# Using the Xarray method open_dataset to load in the netCDF file.

In [ ]:
fpath2='/home/aaron.scott/DATA/BlownUnder/WRF/20200212/'
file=fpath2+'wrfout_d01_2020-02-12_00_00_00.nc'
ds_wrf = xr.open_dataset(file)

In [ ]:
ds_wrf

# With Xarray, you can save off specific data to a variable 

In [ ]:
temp2m = ds_wrf.T2 #sets the 2m temps into a variable - this is a DataArray

In [ ]:
#Type is useful to check the type of your data
type(temp2m)

In [ ]:
#You can check different attributes of your data
temp2m.coords

# Just like Pandas, Xarray has a wrapper around matplotlib! Also notice the use of isel similar to iloc (in Pandas). You can also use loc for label based indexing in Xarray similar to how it is used in Pandas. 

In [ ]:
temp2m[10].plot.pcolormesh()

# Xarray has a useful thing called Faceting. While we won't go into deep detail in this example, below are some examples of how to make nice plots with minimum lines of code!

In [ ]:
temp2m.isel(Time=[0,1,2,3,4]).plot.imshow(col='Time',col_wrap=2,robust=True,cbar_kwargs={'orientation':'vertical','shrink':0.5})

# Below is a standard way of using Matplotlib with Cartopy to make a map.

In [ ]:
lats =temp2m.coords['XLAT'][0,:,:]
lons = temp2m.coords['XLONG'][0,:,:]
projection=ccrs.LambertConformal()


fig = plt.figure(figsize=(12,8))
ax = fig.add_subplot(1,1,1,projection=projection)
ax.set_extent([-108,-92,43,51],ccrs.PlateCarree())
p=ax.contourf(lons,lats,temp2m.isel(Time=9),transform=ccrs.PlateCarree())
fig.colorbar(p)
ax = set_background(ax)

# However, We can use the Xarray wrapper around Matplotlib to do the same thing!

In [ ]:
projection=ccrs.LambertConformal()
fig = plt.figure(figsize=(12,8))
ax = fig.add_subplot(1,1,1,projection=projection)
temp2m.isel(Time=9).plot.pcolormesh(x='XLONG',y='XLAT',ax=ax,transform=ccrs.PlateCarree())
ax = set_background(ax)

# Below is another example with using the Faceting paradigm. 

In [ ]:
map_proj = ccrs.LambertConformal()
p = temp2m.isel(Time=[7,8,9,10]).plot.pcolormesh(x='XLONG',y='XLAT',col='Time',col_wrap=2,
                                 aspect=1,
                                 robust=True,
                                 figsize=(10,10),
                                 transform=ccrs.PlateCarree(),
                                 subplot_kws={'projection':map_proj})

for ax,title in zip(p.axes.flat,list('abcd')):
    ax.set_extent([-108,-92,43,51],ccrs.PlateCarree())
    #ax.set_extent([-100,-95.85,46,49],ccrs.PlateCarree())
    ax = set_background(ax)
    ax.set_title(title)

plt.subplots_adjust(hspace=-0.5, wspace=0.1,top=0.93,bottom=.05,left=0.05,right=.70)


# Here is an example of using Faceting to Plot temperature along a lines of longitude. The syntax here may seem a bit weird, but after some practice this is an extremely efficient way to make quick plots of your data in different ways. 

In [ ]:
temp2m.isel(Time=[8,9],south_north=slice(0,None,10)).plot(x='west_east',hue='south_north',col='Time')

In [ ]:
map_proj = ccrs.LambertConformal()
p = temp2m.plot.pcolormesh(x='XLONG',y='XLAT',col='Time',col_wrap=4,
                                 aspect=1,
                                 robust=True,
                                 
                                 transform=ccrs.PlateCarree(),
                                 subplot_kws={'projection':map_proj})

for ax in p.axes.flat:
    #ax.set_extent([-108,-92,43,51],ccrs.PlateCarree())
    ax.set_extent([-100,-95.85,46,49],ccrs.PlateCarree())
    ax = set_background(ax)

plt.subplots_adjust(hspace=0.3, wspace=0,top=0.93,bottom=.05,left=0.05,right=.70)
